# Google Open Dataset map to SageMaker GroundTruth 

Paths: s3://obj-nonprod-training/google-open/computer-monitor/000060e3121c7305.jpg

calculator --> calculator 

computer-monitor --> computer screen/tablet

earbuds --> earphones 

headphones --> headphones 

mobile-phone --> cell phones 

tablet-computer --> computer screen/tablet


watch --> save for later
                                        
                                        
## CSV Schema 

id: filename without extension 

label: label 

xmin,xmax

ymin,ymax

                     
## TODO: 
- factor in various metaflags such as occlusion/truncated/inside? 

In [1]:
import pandas as pd
import json 
import boto3
from io import BytesIO
from PIL import Image
import numpy as np
from tqdm import tqdm

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

In [2]:
# bucket_name = 'obj-nonprod-training'
# prefix = 'google-open'
dir_map = {'calculator': 3, 
           'computer-monitor': 4, 
           'earbuds': 2,
           'headphones': 1,
           'mobile-phone': 0,
           'tablet-computer': 4
          }

dir_map_flip = {
    0:'Cell phones',
    1:'Headphones',
    2:'Earphones',
    3:'Calculator',
    4:'Computer screen/Tablet'
}

In [3]:
# TODO: Ensure type check for .jpg extension 
go_df = pd.read_csv('../data/go-sub/dropped.csv')

In [4]:
go_df['label'].value_counts()

Computer monitor    5498
Mobile phone        5006
Headphones          1072
Tablet computer      653
Calculator           237
Watch                 17
Name: label, dtype: int64

In [6]:
# go_df.iloc[0]['id']
# >>> M = lambda x: x[0][:-1] if x[0][-1]=='.' else x[0][:]
# >>> df = df.apply(M, axis=1)
go_df.head()

,Unnamed: 0,id,code,group,label,xmin,xmax,ymin,ymax,is_occluded,is_truncated,is_group_of,is_depiction,is_inside
0,0,0000b9115cdf1e54,/m/050k8,train,Mobile phone,0.095703,0.707031,0.497396,0.949219,0,0,0,0,0
1,1,0002a1a755d730ca,/m/050k8,train,Mobile phone,0.178125,0.409375,0.339583,0.695833,1,0,0,0,0
2,2,0005829bfcf77ca3,/m/050k8,train,Mobile phone,0.611875,0.699375,0.585458,0.787535,1,0,0,0,0
3,3,000ad20b5e452b24,/m/050k8,train,Mobile phone,0.000000,0.603125,0.192188,0.917188,0,1,0,0,0
4,4,000ad20b5e452b24,/m/050k8,train,Mobile phone,0.407813,0.998438,0.134375,0.918750,1,1,0,0,0


In [7]:
M = lambda x: x.replace(' ', '-').lower()
go_df['label'] = go_df['label'].apply(M)
go_df.head()

,Unnamed: 0,id,code,group,label,xmin,xmax,ymin,ymax,is_occluded,is_truncated,is_group_of,is_depiction,is_inside
0,0,0000b9115cdf1e54,/m/050k8,train,mobile-phone,0.095703,0.707031,0.497396,0.949219,0,0,0,0,0
1,1,0002a1a755d730ca,/m/050k8,train,mobile-phone,0.178125,0.409375,0.339583,0.695833,1,0,0,0,0
2,2,0005829bfcf77ca3,/m/050k8,train,mobile-phone,0.611875,0.699375,0.585458,0.787535,1,0,0,0,0
3,3,000ad20b5e452b24,/m/050k8,train,mobile-phone,0.000000,0.603125,0.192188,0.917188,0,1,0,0,0
4,4,000ad20b5e452b24,/m/050k8,train,mobile-phone,0.407813,0.998438,0.134375,0.918750,1,1,0,0,0


In [8]:
len(go_df)
# go_df[ go_df['id'] == '09ed05726bd3c3c2' ]

12483

## Different Approaches...

1. From listing images in S3 and access df for labels? 
2. Iterate row by row from df 

In [8]:
# for r in range(len(go_df)): 
#     row = go_df.iloc[r]
#     print(row)
    
#     fname = row['id']+'.jpg'
#     label = row['label']
#     s3_path = f's3://{bucket_name}/{label}/{fname}'
#     print(s3_path)
    
#     class_id = dir_map[label]
    
#     break

In [16]:
# s3://google-open-images/dropped.csv
bucket_name = 'google-open-images'
s3_bucket = s3.Bucket(bucket_name)
prefix = 'train'
job_name = 'go-sub'
job_name_meta = job_name + '-metadata'

In [34]:
with open(f'../data/go-sub/{job_name}.manifest', 'w') as f:
    for obj in s3_bucket.objects.filter(Prefix=f'{prefix}/'):
        if obj.key[-4:] != '.jpg': continue 
            
        gt_row = {}
        gt = {}
        gt_meta = {}

#         print(obj.key)
#         _, label, fpath = obj.key.split('/')
        _, fpath = obj.key.split('/')
        img_id = fpath[:-4]
        
        # Diff way to get label
        label = dir_map_flip[dir_map[go_df [ go_df['id'] == img_id]['label'].tolist()[0]]]
        
        gt_row['source-ref'] = f's3://{bucket_name}/train/{fpath}'

        # Get img dims 
        s3_resp_obj = s3_client.get_object(Bucket=bucket_name, Key=obj.key)
        obj_content = s3_resp_obj['Body'].read()
        img = Image.open(BytesIO(obj_content))
        try: 
            h, w, c = np.array(img).shape # height, width, channels
        except: 
            print(obj.key, np.array(img).shape)
            h, w = np.array(img).shape
            c = 1

        # get boxes 
        img_labels = go_df[ go_df['id'] == img_id ]
        annots = []
        for r in range(len(img_labels)):
            row = img_labels.iloc[r]
            box = {}
            
            if row['label'] == 'watch': 
                print('WATCH')
                break 

            # boxes are top left origin
            box['class_id'] = int(dir_map[row['label']])+1 if label=='earbuds' else int(dir_map[row['label']])
            box['width'] = int((row['xmax'] - row['xmin']) * w)
            box['top'] = int(row['ymin']*h)
            box['height'] = int((row['ymax'] - row['ymin']) * h)
            box['left'] = int(row['xmin']*w)

            annots.append(box)

        img_size = [{
            'width': w,
            'depth': c,
            'height': h
        }]

        gt['annotations'] = annots 
        gt['image_size'] = img_size

        # generate class-map
        cmap = {}
        for b in annots: 
            c = b['class_id']
            if c in dir_map.values():
                cmap[str(c)] = dir_map_flip[c]
        gt_meta['class-map'] = cmap

        # generate object confidence 
        obj_conf = []
        for i in range(len(annots)): 
            obj_conf.append({"confidence": 0})
        gt_meta['objects'] = obj_conf

        # basic string metadata fields 
        gt_meta['job-name'] = 'labeling-job/go-sub'
        gt_meta['human-annotated'] = 'yes'
        gt_meta['creation-date'] = "2020-08-13T18:27:33.915000"
        gt_meta['type'] = 'groundtruth/object-detection'

        gt_row[job_name] = gt 
        gt_row[job_name_meta] = gt_meta

#         if len(annots)>1:
#         print(json.dumps(gt_row, indent=2))

#         break 

        # write to a file -- ONLY if annotations 
        if len(annots)>0: 
            f.write(json.dumps(gt_row) + '\n')
        else: 
            print(f'Skipping {obj.key} bc annotations empty: {annots}')

train/0209ead7ea5b4549.jpg (683, 1024)
train/03521e09321cf595.jpg (679, 1024)
train/03bd898e00808f12.jpg (854, 1024)
train/0a36d7f97849c69a.jpg (1024, 891)
train/0b4a0adbd7929fff.jpg (731, 1024)
train/0bfdef17703e0f1f.jpg (768, 1024)
train/102fa5190041d51b.jpg (678, 1024)
train/173eae3087f84a07.jpg (681, 1024)
WATCH
train/23369e2b6ede5fea.jpg (1024, 680)
train/241e77f2f14c82ca.jpg (986, 1024)
train/2a7a4cff36b7af86.jpg (1024, 768)
WATCH
train/41fc7a303a5e4bc4.jpg (1024, 692)
train/47b32d9b609933af.jpg (768, 1024)
train/50fbcc86523f446e.jpg (1024, 1024)
train/5d2d6650c1dfd4ae.jpg (1024, 1024)
train/6eb03384f25d3f01.jpg (423, 1024)
train/99a4d5e2a030ddd5.jpg (1024, 683)
train/9f6df44cb0298f56.jpg (854, 1024)
train/a951103df349d405.jpg (682, 1024)
train/bab27086ac2767b4.jpg (1024, 1024)
WATCH
train/bf11cc7272a9b5a0.jpg (768, 1024)
train/c9eb80b2c7d76157.jpg (682, 1024)
train/cddd935cb51493d7.jpg (683, 1024)
train/d270d230201e96dc.jpg (1024, 1024)
train/d40a8fda04c59bf9.jpg (768, 1024)
WAT

In [35]:
print(1)

1


In [30]:
# dir_map_flip[dir_map[go_df [ go_df['id'] == '000060e3121c7305']['label'].tolist()[0]]]

In [33]:
go_df [ go_df['id'] == '04017f80b149f4eb']

,Unnamed: 0,id,code,group,label,xmin,xmax,ymin,ymax,is_occluded,is_truncated,is_group_of,is_depiction,is_inside
4882,4882,04017f80b149f4eb,/m/02522,train,computer-monitor,0.46125,0.698125,0.401316,0.674812,1,0,0,0,0
